# GPAW

In [ ]:
from ase import Atoms
from gpaw import GPAW
import numpy as np

import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams['font.size'] = 20

In [ ]:
basepath = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/GPAW/'
# cell parameters
boxsize = 10.0
d = 1.12
pos1 = [boxsize/2, boxsize/2, boxsize/2-d/2]
pos2 = [boxsize/2, boxsize/2, boxsize/2+d/2]
molecules = ['H2', 'He2', 'Li2', 'Be2', 'B2', 'C2', 'N2']

In [ ]:
for mol in molecules:
    molecule = Atoms(mol, positions=[pos1, pos2], cell=(boxsize, boxsize, boxsize), pbc=False)
    calc = GPAW(mode='fd', h = 0.1, xc='LDA', nbands=-4)
    molecule.set_calculator(calc)
    molecule.get_total_energy()
    n = calc.get_all_electron_density()
    ev = calc.get_eigenvalues()
    hl = calc.get_homo_lumo()
    occ = calc.get_occupation_numbers()
    
    np.save(basepath+mol+'_dens', n)
    np.savetxt(basepath+mol+'_ev', ev)
    np.savetxt(basepath+mol+'_hl', hl)
    np.savetxt(basepath+mol+'_occ', occ)
    

In [ ]:
# test

In [ ]:
molecule = Atoms(molecules[1], positions=[pos1, pos2], cell=(boxsize, boxsize, boxsize), pbc=False)
calc = GPAW(mode='fd', h = 0.5, xc='LDA', txt='H2.txt', nbands=-4)
molecule.set_calculator(calc)
molecule.get_total_energy()

In [ ]:
n = calc.get_all_electron_density()
n_ps = calc.get_pseudo_density()

In [ ]:
plt.plot(n.sum(axis=(0,1)))
plt.plot(n_ps.sum(axis=(0,1)))

In [ ]:
n.shape[0]

In [ ]:
1/n.shape[0]**3*n.sum()*1000

In [ ]:
calc.get_eigenvalues()

In [ ]:
calc.get_homo_lumo()

In [ ]:
calc.get_occupation_numbers()

# VASP

In [ ]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')

import alchemy_tools2 as at
import numpy as np
from explore_qml_data import get_free_atom_data

import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams['font.size'] = 20

In [ ]:
basepath = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/'
subpath = ['H2', 'He2', 'Li2', 'Be2', 'B2', 'C2', 'N2']
paths = []
for sp in subpath:
    paths.append(basepath + sp + '/' + 'CHG')


# $\partial_{\lambda} E$

In [ ]:
densities, nuclei, gpts, h_matrix = at.load_vasp_dens(paths)

In [ ]:
lam_vals = np.array([0,1,2,3,4,5,6,7])/7

In [ ]:
nuclei, atomic_energies_with_repulsion, atomic_energies, alch_pots = at.atomic_energy_decomposition(lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
dEdlam = []
for d in densities:
    aer, atomic_energies, alch_pots = at.calculate_atomic_energies(d, nuclei, gpts, h_matrix)
    dEdlam.append(atomic_energies[0])
dEdlam = np.array(dEdlam)

In [ ]:
# # without electronic potential
# fig, ax = plt.subplots(1,1)
# plt.plot(lam_vals, dEdlam, '-o')
# ax.set_xlabel(r'$\lambda$')
# ax.set_ylabel(r'$\partial_{\lambda} E$')

In [ ]:
e_lumo = {'non':0.0, 'H2':-1.2080, 'He2':-0.1406 , 'Li2':-3.2962, 'Be2': -4.4627, 'B2':-6.1195, 'C2':-7.9358, 'N2':-2.0942}
# e_homo = {'H2':-8.8894, 'He2':}

In [ ]:
dElam_epot = dEdlam.copy()
for i,k in enumerate(e_lumo):
    dElam_epot[i] += e_lumo[k]
    print(e_lumo[k])

In [ ]:
# without electronic potential
fig, ax = plt.subplots(1,1)
plt.plot(lam_vals, dEdlam, '-o', label='without electronic potential')
plt.plot(lam_vals, dElam_epot, '-o', label='with electronic potential')
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\partial_{\lambda} E$')
ax.legend()